In [ ]:
import numpy as np
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
from tqdm import tqdm
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

def g0(Z_t, C):
    return np.cos(Z_t @ C)# 假设是这个函数,cosine

def make_odefunc(beta_0, Z_interp, C):
    def odefunc(t, X_flat):
        X = X_flat.reshape(1, -1)
        Z_t = Z_interp(t).reshape(1, -1)
        return (X @ beta_0 + g0(Z_t, C)).flatten()
    return odefunc

def find_best_alpha(Z, Y, alphas, cv):
    """
    Use GridSearchCV to find the best alpha for Kernel Ridge Regression.
    """
    model = KernelRidge(kernel='rbf')
    param_grid = {'alpha': alphas}
    grid = GridSearchCV(model, param_grid, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)
    grid.fit(Z, Y)

    best_alpha = grid.best_params_['alpha']
    best_mse = -grid.best_score_  # 注意：scoring 是 neg_MSE，要取负号
    return best_alpha

In [ ]:

for i in range(200):
  Z = np.random.randn(T, nu)
  Z_interp = interp1d(t_grid, Z, axis=0, kind='cubic', fill_value="extrapolate")  # Interpolate Z(t) for continuous time within the solver
  
  X0 = np.zeros(p)
  
  C = np.random.randn(nu, p)
  
  odefunc = make_odefunc(beta_0, Z_interp, C)
  sol = solve_ivp(odefunc, [0, 1], X0, t_eval=t_grid, method='RK45')
  X_true = sol.y.T  # (T,p)
  
  dXdt = np.zeros_like(X_true)
  for idx, t in enumerate(t_grid):
      X_t = X_true[idx, :].reshape(1, -1)
      Z_t = Z[idx, :].reshape(1, -1)
      dXdt[idx, :] = (X_t @ beta_0 + g0(Z_t, C)).flatten()
      
  
  #更改了U的噪声强度从0.05到0.1
  U = 0.1 * np.random.randn(T, p)  # noise
  Y = X_true + U  # observation
  
  
  Q = 20
  kf = KFold(n_splits=Q, shuffle=True, random_state=42)
  beta_q_list = []
  Delta_t = t_grid[1] - t_grid[0]
  
  
  best_alpha = find_best_alpha(Z, Y, alphas = np.logspace(-2,2,20), cv= KFold(n_splits=5, shuffle=True, random_state=42))
  
  for Iqc, Iq in tqdm(kf.split(range(T)), total=Q, desc="Cross-fitting"):
      # Estimate r_hat, g_hat on Iqc (complement of chunk q)
      Z_qc, Y_qc = Z[Iqc], Y[Iqc]
      Z_q, Y_q = Z[Iq], Y[Iq]
      dXdt_q = dXdt[Iq]
  
      r_hat = KernelRidge(kernel='rbf', alpha=best_alpha)  # Estimate r_hat with kernel ridge regression
      r_hat.fit(Z_qc, Y_qc)
      r_hat_q = r_hat.predict(Z_q)
      
      g_hat = RandomForestRegressor(n_estimators=100) #change into random forest regressor
                          
     
      #g_hat = RandomForestRegressor(n_estimators=100)
      g_hat.fit(Z_qc, dXdt[Iqc].ravel())
      g_hat_q = g_hat.predict(Z_q)
  
      # Solve for beta_hat using Neyman orthogonal score
      d_l = r_hat_q.flatten() - Y_q.flatten()  # shape (len(Iq),)
      s_l = np.cumsum(Y_q.flatten()) * Delta_t  # integrate X using observed Y
      g_l = np.cumsum(g_hat_q) * Delta_t  # integrate g_hat
  
      beta_hat_q = np.sum(d_l * (r_hat_q.flatten() - g_l)) / np.sum(d_l * s_l)#这里用r_hat_q估计效果更好，why
      beta_q_list.append(beta_hat_q)
  
  # Average over Q chunks
  beta_hat = np.mean(beta_q_list)
  
  print("Estimated beta:\n", beta_hat)
  beta_forest.append(beta_hat)
